In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import os
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics
import math
import datetime
import warnings
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import xgboost as xgb
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from ipywidgets import HBox, VBox
from tabulate import tabulate
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance, plot_tree
#Matplotlib style
plt.style.use('fivethirtyeight')
#Ignoring some warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'plotly'

In [ ]:
inputs = pd.read_csv(r'../Data/all_inputs_cleaned.csv')
outputs = pd.read_csv(r'../Data/all_outputs_cleaned.csv')

# Prediction for Construction

In [ ]:
fig = px.line(outputs['Construction'], width=1000, height=500)
fig.update_layout(title_text='Monthly employment data of Construction from 2006-06-01 to 2023-09-01')
fig.show()

In [ ]:
#This code uses the pacf() function from the tsa.stattools module of the statsmodels library (sm) to compute the autocorrelation function.
plot_pacf(outputs['Construction'],method="yw")
#Print the visualization
plt.show()

In [ ]:
const_data = outputs[['Month','Construction']]
#Convert the date column to datetime format
const_data["Month"] = pd.to_datetime(const_data["Month"])
for i in range(1,5):
    const_data['prev'+str(i)]=const_data['Construction'].shift(i)
train_ml, test_ml, y_train, y_test = train_test_split(const_data, const_data['Construction'], test_size=0.3, random_state=0)

In [ ]:
N = len(inputs.keys())
for i in range(2,N):
    key = inputs.keys()[i]
    for i in range(1,6):
        inputs[key + '_prev' + str(i)] = inputs[key].shift(i)
N = len(inputs.keys())
inputs.head()

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['quarter'] = df['Month'].dt.quarter
    df['month'] = df['Month'].dt.month
    df['year'] = df['Month'].dt.year
    for i in range(2,N):
        df[inputs.keys()[i]] = inputs.iloc[:,i]
    features = ['quarter','month','year']
    input_feat = ['CPI', 'InterestRate', 'GDP','Borrowing', 'ITBVol', 'VGTVol']
    for feature in input_feat:
        for i in range(1,5):
            features.append(feature+'_prev'+str(i))
    for i in range(1,5):
        features.append('prev'+str(i))
#     features = ['quarter','month','year', 'prev1', 'prev3', 'prev4', 'InterestRate_prev2', 'Borrowing_prev3', 'Borrowing_prev4','VGTVol_prev3', 'VGTVol_prev4', 'CPI_prev1', 'CPI_prev2', 'CPI_prev4', 'ITBVol_prev1']
    X = df[features]
#     X = df[['quarter','month','year','CPI', 'InterestRate', 'GDP', 'ValAddConst', 'ValAddInfo','Borrowing','CommercialLoan', 'ConsumerLoan','Deficit','ITBPrice', 'ITBVol','VGTPrice', 'VGTVol','S&P500Price','S&P500Vol']]
    # X = df[['month','year','CPI', 'GDP', 'ValAddConst','Borrowing','Deficit','ITBPrice','VGTPrice', 'VGTVol']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
# Creating the features for the train and test sets
X_train, y_train = create_features(train_ml, label="Construction")
X_test, y_test = create_features(test_ml, label="Construction")
X_train.head()

In [ ]:
#Defining and fitting the model
reg = xgb.XGBRegressor(n_estimators=1000,early_stopping_rounds=60,)
# reg = xgb.XGBRegressor(
#     n_estimators=200, 
#     learning_rate=0.009, 
#     max_depth=5, 
#     min_child_weight=1,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     reg_lambda=1,  # Corrected parameter name
#     alpha=0,
# )
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
       verbose=False)

In [ ]:
#Plot of feature importance
_ = plot_importance(reg, height=0.9)

In [ ]:
#Predicting with our model for both the train and test data
train_ml["Predictions"] = reg.predict(X_train)
test_ml['Prediction'] = reg.predict(X_test)

In [ ]:
#Plotting the predictions of the training data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(train_ml["Predictions"]),list(train_ml["Construction"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Construction",0:"Predicted Construction"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Construction TRAIN', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
#Plotting the predictions of the testing data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(test_ml["Prediction"]),list(test_ml["Construction"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Construction",0:"Predicted Construction"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Construction TEST', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
pred_sign = np.sign(train_ml['Predictions'])
y_test_sign = np.sign(train_ml['Construction'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
pred_sign = np.sign(test_ml['Prediction'])
y_test_sign = np.sign(test_ml['Construction'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

Metrics:


Root Mean Square Error, RMSE: Square root of the mean of the difference between the actual data points and the squared prediction value. It penalizes greater or extreme differences more.

Mean Absolute Percentage Error, MAPE: It allows to measure errors relative to the magnitude of the real value.

In [ ]:
RMSE_Serie_XG = mean_squared_error(train_ml["Construction"],train_ml["Predictions"],squared=False)
MAPE_Serie_XG = mean_absolute_percentage_error(train_ml["Construction"], train_ml["Predictions"])
print(f"The RMSE of our XGBOOST model in the full serie data is {round(RMSE_Serie_XG,4)}")
print(f"The MAPE of our XGBOOST model in the full serie data is {round(MAPE_Serie_XG*100,2)}%")

In [ ]:
true_vol = test_ml['Prediction']
pred_vol = test_ml["Construction"]
RMSE_XG = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE_XG = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our XGBOOST model in the predicted data is {round(RMSE_XG,4)}")
print(f"The MAPE of our XGBOOST model in the predicted data is {round(MAPE_XG*100,2)}%")

In [ ]:
X, y = create_features(const_data, label="Construction")
const_data["Predictions"] = reg.predict(X)
#Plotting the predictions of the training data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(const_data["Predictions"]),list(const_data["Construction"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Construction",0:"Predicted Construction"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Construction FULL', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
pred_sign = np.sign(const_data['Predictions'])
y_test_sign = np.sign(const_data['Construction'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
true_vol = const_data['Predictions']
pred_vol = const_data["Construction"]
RMSE_XG = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE_XG = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our XGBOOST model in the predicted data is {round(RMSE_XG,4)}")
print(f"The MAPE of our XGBOOST model in the predicted data is {round(MAPE_XG*100,2)}%")
r2_score = sklearn.metrics.r2_score(true_vol, pred_vol)
print(f"The R^2 score of our XGBOOST model in the predicted data is {round(r2_score,4)}")

# Prediction for Information

In [ ]:
fig = px.line(outputs['Information'], width=1000, height=500)
fig.update_layout(title_text='Monthly employment data of Information from 2006-06-01 to 2023-09-01')
fig.show()

In [ ]:
plot_pacf(outputs['Information'],method="yw")
#Print the visualization
plt.show()

In [ ]:
info_data = outputs[['Month','Information']]
#Convert the date column to datetime format
info_data["Month"] = pd.to_datetime(info_data["Month"])
for i in range(1,6):
    info_data['prev'+str(i)]=info_data['Information'].shift(i)
train_ml, test_ml, y_train, y_test = train_test_split(info_data, info_data['Information'], test_size=0.35, random_state=0)
train_ml.head()

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['quarter'] = df['Month'].dt.quarter
    df['month'] = df['Month'].dt.month
    df['year'] = df['Month'].dt.year
    for i in range(2,N):
        df[inputs.keys()[i]] = inputs.iloc[:,i]
    features = ['quarter','month','year']
    input_feat = ['CPI', 'InterestRate', 'GDP','Borrowing', 'ITBVol', 'VGTVol']
    for feature in input_feat:
        for i in range(1,6):
            features.append(feature+'_prev'+str(i))
#     features = ['quarter','month','year', 'prev1', 'prev3', 'prev4', 'InterestRate_prev2', 'Borrowing_prev3', 'Borrowing_prev4','VGTVol_prev3', 'VGTVol_prev4', 'CPI_prev1', 'CPI_prev2', 'CPI_prev4', 'ITBVol_prev1']
#     X = df[features]
    # X = df[['quarter','month','year']]
    # X = df[['quarter','month','year','CPI', 'InterestRate', 'GDP', 'ValAddConst', 'ValAddInfo','Borrowing','CommercialLoan', 'ConsumerLoan','Deficit','ITBPrice', 'ITBVol','VGTPrice', 'VGTVol','S&P500Price','S&P500Vol']]
    # X = df[['quarter','CPI', 'InterestRate','CommercialLoan', 'ConsumerLoan','Deficit','ITBPrice', 'VGTVol','S&P500Price','S&P500Vol']]
#     X = df[['quarter','month','year','CPI', 'InterestRate', 'GDP','Borrowing','CommercialLoan', 'ConsumerLoan','Deficit','ITBPrice','VGTPrice','S&P500Price']]
#     features = ['quarter','year','CPI', 'InterestRate', 'GDP','CommercialLoan', 'ConsumerLoan','ITBPrice', 'ITBVol', 'VGTVol']
    for i in range(1,6):
        features.append('prev'+str(i))
    X = df[features]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
#Creating the features for the train and test sets
X_train, y_train = create_features(train_ml, label="Information")
X_test, y_test = create_features(test_ml, label="Information")
X_train.head()

In [ ]:
#Defining and fitting the model
reg = xgb.XGBRegressor(n_estimators=1000,early_stopping_rounds=60,)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
       verbose=False)

In [ ]:
#Plot of feature importance
_ = plot_importance(reg, height=0.9)

In [ ]:
#Predicting with our model for both the train and test data
train_ml["Predictions"] = reg.predict(X_train)
test_ml['Prediction'] = reg.predict(X_test)

In [ ]:
#Plotting the predictions of the training data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(train_ml["Predictions"]),list(train_ml["Information"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Information",0:"Predicted Information"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Information TRAIN', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
#Plotting the predictions of the testing data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(test_ml["Prediction"]),list(test_ml["Information"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Information",0:"Predicted Information"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Information TEST', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
pred_sign = np.sign(train_ml['Predictions'])
y_test_sign = np.sign(train_ml['Information'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
pred_sign = np.sign(test_ml['Prediction'])
y_test_sign = np.sign(test_ml['Information'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
RMSE_Serie_XG = mean_squared_error(train_ml["Information"],train_ml["Predictions"],squared=False)
MAPE_Serie_XG = mean_absolute_percentage_error(train_ml["Information"], train_ml["Predictions"])
print(f"The RMSE of our XGBOOST model in the full serie data is {round(RMSE_Serie_XG,4)}")
print(f"The MAPE of our XGBOOST model in the full serie data is {round(MAPE_Serie_XG*100,2)}%")

In [ ]:
true_vol = test_ml['Prediction']
pred_vol = test_ml["Information"]
RMSE_XG = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE_XG = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our XGBOOST model in the predicted data is {round(RMSE_XG,4)}")
print(f"The MAPE of our XGBOOST model in the predicted data is {round(MAPE_XG*100,2)}%")

In [ ]:
X, y = create_features(info_data, label="Information")
info_data["Predictions"] = reg.predict(X)
#Plotting the predictions of the training data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(info_data["Predictions"]),list(info_data["Information"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Information",0:"Predicted Information"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Information FULL', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
pred_sign = np.sign(info_data['Predictions'])
y_test_sign = np.sign(info_data['Information'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
true_vol = info_data['Predictions']
pred_vol = info_data["Information"]
RMSE_XG = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE_XG = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our XGBOOST model in the predicted data is {round(RMSE_XG,4)}")
print(f"The MAPE of our XGBOOST model in the predicted data is {round(MAPE_XG*100,2)}%")

# Prediction for Total Private

In [ ]:
fig = px.line(outputs['Total_Private'], width=1000, height=500)
fig.update_layout(title_text='Monthly employment data of Total_Private from 2006-06-01 to 2023-09-01')
fig.show()

In [ ]:
plot_pacf(outputs['Total_Private'],method="yw")
#Print the visualization
plt.show()

In [ ]:
totpriv_data = outputs[['Month','Total_Private']]
#Convert the date column to datetime format
totpriv_data["Month"] = pd.to_datetime(totpriv_data["Month"])
for i in range(1,6):
    totpriv_data['prev'+str(i)]=totpriv_data['Total_Private'].shift(i)
train_ml, test_ml, y_train, y_test = train_test_split(totpriv_data, totpriv_data['Total_Private'], test_size=0.3, random_state=4)

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['quarter'] = df['Month'].dt.quarter
    df['month'] = df['Month'].dt.month
    df['year'] = df['Month'].dt.year
    for i in range(2,N):
        df[inputs.keys()[i]] = inputs.iloc[:,i]
    features = ['year']
    input_feat = ['CommercialLoan','VGTPrice','CPI']
    for feature in input_feat:
        for i in range(1,3):
            features.append(feature+'_prev'+str(i))
    for i in range(1,3):
        features.append('prev'+str(i))
    X = df[features]
    # X = df[['quarter','month','year']]
#     X = df[['year','CommercialLoan','VGTPrice', 'CPI']]
    # X = df[['quarter','month','year','CPI', 'InterestRate', 'GDP', 'ValAddConst', 'ValAddInfo','Borrowing','CommercialLoan', 'ConsumerLoan','Deficit','ITBPrice', 'ITBVol','VGTPrice', 'VGTVol','S&P500Price','S&P500Vol']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
#Creating the features for the train and test sets
X_train, y_train = create_features(train_ml, label="Total_Private")
X_test, y_test = create_features(test_ml, label="Total_Private")
X_train.head()

In [ ]:
#Defining and fitting the model
reg = xgb.XGBRegressor(n_estimators=1000,early_stopping_rounds=60,)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
       verbose=False)

In [ ]:
#Plot of feature importance
_ = plot_importance(reg, height=0.9)

In [ ]:
#Predicting with our model for both the train and test data
train_ml["Predictions"] = reg.predict(X_train)
test_ml['Prediction'] = reg.predict(X_test)

In [ ]:
#Plotting the predictions of the training data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(train_ml["Predictions"]),list(train_ml["Total_Private"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Total_Private",0:"Predicted Total_Private"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Total_Private TRAIN', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
#Plotting the predictions of the testing data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(test_ml["Prediction"]),list(test_ml["Total_Private"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Total_Private",0:"Predicted Total_Private"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Total_Private TEST', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
pred_sign = np.sign(train_ml['Predictions'])
y_test_sign = np.sign(train_ml['Total_Private'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
pred_sign = np.sign(test_ml['Prediction'])
y_test_sign = np.sign(test_ml['Total_Private'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
RMSE_Serie_XG = mean_squared_error(train_ml["Total_Private"],train_ml["Predictions"],squared=False)
MAPE_Serie_XG = mean_absolute_percentage_error(train_ml["Total_Private"], train_ml["Predictions"])
print(f"The RMSE of our XGBOOST model in the full serie data is {round(RMSE_Serie_XG,4)}")
print(f"The MAPE of our XGBOOST model in the full serie data is {round(MAPE_Serie_XG*100,2)}%")

In [ ]:
true_vol = test_ml['Prediction']
pred_vol = test_ml["Total_Private"]
RMSE_XG = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE_XG = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our XGBOOST model in the predicted data is {round(RMSE_XG,4)}")
print(f"The MAPE of our XGBOOST model in the predicted data is {round(MAPE_XG*100,2)}%")

In [ ]:
X, y = create_features(totpriv_data, label="Total_Private")
totpriv_data["Predictions"] = reg.predict(X)
#Plotting the predictions of the training data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(totpriv_data["Predictions"]),list(totpriv_data["Total_Private"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Total_Private",0:"Predicted Total_Private"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs Total_Private FULL', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
pred_sign = np.sign(totpriv_data['Predictions'])
y_test_sign = np.sign(totpriv_data['Total_Private'])
classification_accuracy = 1 - 0.5*np.sum(np.abs(pred_sign - y_test_sign))/np.size(y_test_sign)
classification_accuracy

In [ ]:
true_vol = totpriv_data['Predictions']
pred_vol = totpriv_data["Total_Private"]
RMSE_XG = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE_XG = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our XGBOOST model in the predicted data is {round(RMSE_XG,4)}")
print(f"The MAPE of our XGBOOST model in the predicted data is {round(MAPE_XG*100,2)}%")
r2_score = sklearn.metrics.r2_score(true_vol, pred_vol)
print(f"The R^2 score of our XGBOOST model in the predicted data is {round(r2_score,4)}")